# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-27-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-27-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-28 04:28:11,33.93911,67.709953,38129,1401,29046,7682.0,Afghanistan,97.946635,3.674369
1,NaN,NaN,NaN,Albania,2020-08-28 04:28:11,41.15330,20.168300,9083,266,4791,4026.0,Albania,315.623045,2.928548
2,NaN,NaN,NaN,Algeria,2020-08-28 04:28:11,28.03390,1.659600,43016,1475,30157,11384.0,Algeria,98.095728,3.428957
3,NaN,NaN,NaN,Andorra,2020-08-28 04:28:11,42.50630,1.521800,1098,53,893,152.0,Andorra,1421.083285,4.826958
4,NaN,NaN,NaN,Angola,2020-08-28 04:28:11,-11.20270,17.873900,2415,105,1335,975.0,Angola,7.347959,4.347826


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37599,37599,37856,37894,37953,37999,38054,38070,38113,38129
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7654,7812,7967,8119,8275,8427,8605,8759,8927,9083
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,39444,39847,40258,40667,41068,41460,41858,42228,42619,43016


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1375,1375,1385,1385,1385,1387,1389,1397,1401,1401
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,232,234,238,240,245,250,254,259,263,266
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1391,1402,1411,1418,1424,1435,1446,1456,1465,1475


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,27166,27166,27681,28016,28016,28180,28360,28440,29042,29046
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3871,3928,3986,4096,4184,4332,4413,4530,4633,4791
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,27653,27971,28281,28587,28874,29142,29369,29587,29886,30157


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1235,1241,1240,1255,1264,1266,1286,1286,1281,1284
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3906,3931,3957,3997,4037,4069,4083,4100,4166,4199
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,606,613,619,622,626,629,633,628,616,614


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,22,22,22,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,30,30,31,32,32,32,32,32,33,34
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-28 04:28:11,32.539527,-86.644082,1284,22,0,1262.0,"Autauga, Alabama, US",2298.233367,1.713396
668,1075.0,Lamar,Alabama,US,2020-08-28 04:28:11,33.779950,-88.096680,266,2,0,264.0,"Lamar, Alabama, US",1926.838102,0.751880
669,1077.0,Lauderdale,Alabama,US,2020-08-28 04:28:11,34.901719,-87.656247,1373,22,0,1351.0,"Lauderdale, Alabama, US",1480.658694,1.602331


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5867785,180824,2101326
Brazil,3761391,118649,3122846
India,3387500,61529,2583948


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5867785,180824,2101326,3585635,2020-08-28 04:28:11,37.919625,-91.366793
Brazil,3761391,118649,3122846,519896,2020-08-28 04:28:11,-12.669522,-48.480493
India,3387500,61529,2583948,742023,2020-08-28 04:28:11,23.088275,81.806127
Russia,972972,16758,790629,165585,2020-08-28 04:28:11,54.546312,62.120860
South Africa,618286,13628,531338,73320,2020-08-28 04:28:11,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,691821,12677,0
Texas,614549,12365,0
Florida,611991,10868,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,691821,12677,0,679144,2020-08-28 04:28:11,37.843962,-120.728594
Texas,614549,12365,0,602184,2020-08-28 04:28:11,31.660643,-98.653069
Florida,611991,10868,0,601123,2020-08-28 04:28:11,28.940755,-82.700744
New York,432131,32926,0,399205,2020-08-28 04:28:11,42.671593,-75.579694
Georgia,263074,5393,0,257681,2020-08-28 04:28:11,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,236986,5701,0
New York,New York City,233140,23674,0
Florida,Miami-Dade,154757,2346,0
Arizona,Maricopa,132704,2896,0
Illinois,Cook,123691,5039,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,236986,5701,0,231285,2020-08-28 04:28:11,34.308284,-118.228241,6037.0
New York,New York City,233140,23674,0,209466,2020-08-28 04:28:11,40.767273,-73.971526,36061.0
Florida,Miami-Dade,154757,2346,0,152411,2020-08-28 04:28:11,25.611236,-80.551706,12086.0
Arizona,Maricopa,132704,2896,0,129808,2020-08-28 04:28:11,33.348359,-112.491815,4013.0
Illinois,Cook,123691,5039,0,118652,2020-08-28 04:28:11,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,38070,8759,42228,1060,2283,94,359638,42936,25204,25706,...,329821,1536,39641,41158,1029,19678,10,1924,11285,6196
2020-08-26,38113,8927,42619,1098,2332,94,370188,43067,25322,26033,...,330967,1543,39964,41965,1034,20155,10,1930,11376,6251
2020-08-27,38129,9083,43016,1098,2415,94,380292,43270,25448,26361,...,332509,1551,40447,42898,1036,20677,10,1933,11601,6292


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,1397,259,1456,53,102,3,7563,858,549,733,...,41535,43,284,343,27,133,1,557,282,166
2020-08-26,1401,263,1465,53,103,3,7839,861,572,733,...,41552,43,291,351,30,137,1,560,282,179
2020-08-27,1401,266,1475,53,105,3,8050,864,583,733,...,41564,43,298,358,30,141,1,562,282,189


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,28440,4530,29587,877,977,89,263202,36475,20098,21888,...,1551,1309,35869,32015,592,13162,8,1091,10400,4961
2020-08-26,29042,4633,29886,893,977,89,268801,36726,20099,22145,...,1551,1322,36402,32931,632,13929,8,1097,10693,5001
2020-08-27,29046,4791,30157,893,1335,89,274458,36988,20365,22317,...,1564,1333,37200,34147,637,14194,8,1098,10840,5010


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1235,1241,1240,1255,1264,1266,1286,1286,1281,1284
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,247,249,255,259,259,260,261,264,266,266
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1269,1278,1287,1308,1320,1329,1339,1353,1364,1373
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,377,379,383,389,396,402,411,419,427,432
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2861,2888,2899,2908,2916,2930,2940,2946,3006,3031


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-25,117242,4843,0,199273,57374,681032,55780,52040,16962,49,...,145417,604328,49767,1572,998,114626,71705,9399,71492,3632
2020-08-26,119254,4895,0,199459,58023,687004,55973,52220,16986,49,...,147353,610648,50174,1577,1030,115448,72161,9544,72260,3680
2020-08-27,121023,4974,0,200139,58745,691821,56323,52350,16976,49,...,149179,614549,50557,1586,1052,116569,72703,9633,73138,3733


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-25        1286    4100     628  511    967     527    804    2179   
2020-08-26        1281    4166     616  513    968     526    809    2215   
2020-08-27        1284    4199     614  515    979     527    813    2225   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-25          863      334  ...     158      7      124       48   
2020-08-26          862      331  ...     160      7      127       48   
2020-08-27          860      336  ...     160      7      127       48   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-25            295   412   283          0      108     11  
2020-08-26            295   417   289          0      108     11  
2020-08-27            295   417   289         53      108     11  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-25,2037,36,0,4792,711,12382,1926,4463,603,0,...,1628,11935,397,58,12,2494,1876,187,1094,37
2020-08-26,2045,37,0,4896,732,12532,1927,4463,604,0,...,1648,12140,401,58,14,2515,1880,190,1100,37
2020-08-27,2076,37,0,4929,739,12677,1931,4465,604,0,...,1673,12365,403,58,14,2527,1890,198,1111,37


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-25          22      32       7    6      6      13     36      24   
2020-08-26          22      33       7    5      7      13     36      25   
2020-08-27          22      34       7    5      7      13     36      26   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-25           38        9  ...       0      0        0        0   
2020-08-26           38        9  ...       0      0        0        0   
2020-08-27           38       11  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-25              0     0     0         36        0      0  
2020-08-26              0     0     0         36        0      0  
2020-08-27              0     0     0         36        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,0.000420,0.017897,0.008839,0.000000,0.027453,0.0,0.024998,0.002592,0.006027,0.008276,...,0.003655,0.001957,0.007446,0.020328,0.006849,0.024202,0.0,0.004175,0.012289,0.020758
2020-08-26,0.001129,0.019180,0.009259,0.035849,0.021463,0.0,0.029335,0.003051,0.004682,0.012721,...,0.003475,0.004557,0.008148,0.019607,0.004859,0.024240,0.0,0.003119,0.008064,0.008877
2020-08-27,0.000420,0.017475,0.009315,0.000000,0.035592,0.0,0.027294,0.004714,0.004976,0.012599,...,0.004659,0.005185,0.012086,0.022233,0.001934,0.025899,0.0,0.001554,0.019778,0.006559


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,0.005760,0.019685,0.006916,0.0,0.020000,0.0,0.026745,0.004684,0.045714,0.0,...,0.000385,0.02381,0.014286,0.017804,0.000000,0.000000,0.0,0.003604,0.007143,0.070968
2020-08-26,0.002863,0.015444,0.006181,0.0,0.009804,0.0,0.036493,0.003497,0.041894,0.0,...,0.000409,0.00000,0.024648,0.023324,0.111111,0.030075,0.0,0.005386,0.000000,0.078313
2020-08-27,0.000000,0.011407,0.006826,0.0,0.019417,0.0,0.026917,0.003484,0.019231,0.0,...,0.000289,0.00000,0.024055,0.019943,0.000000,0.029197,0.0,0.003571,0.000000,0.055866


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,0.002821,0.026513,0.007423,0.000000,0.114025,0.0,0.024974,0.011817,0.025356,0.010666,...,0.000645,0.010811,0.008945,0.030084,0.008518,0.108846,0.0,0.000917,0.018809,0.002222
2020-08-26,0.021167,0.022737,0.010106,0.018244,0.000000,0.0,0.021273,0.006881,0.000050,0.011742,...,0.000000,0.009931,0.014860,0.028612,0.067568,0.058274,0.0,0.005500,0.028173,0.008063
2020-08-27,0.000138,0.034103,0.009068,0.000000,0.366428,0.0,0.021045,0.007134,0.013234,0.007767,...,0.008382,0.008321,0.021922,0.036926,0.007911,0.019025,0.0,0.000912,0.013747,0.001800


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-25,0.004558,0.006861,NaN,0.004329,0.008437,0.008098,0.008297,0.000558,0.001180,0.0,...,0.005622,0.010139,0.008164,0.003831,0.014228,0.008827,0.004680,0.009018,0.009004,0.008049
2020-08-26,0.017161,0.010737,NaN,0.000933,0.011312,0.008769,0.003460,0.003459,0.001415,0.0,...,0.013313,0.010458,0.008178,0.003181,0.032064,0.007171,0.006359,0.015427,0.010742,0.013216
2020-08-27,0.014834,0.016139,NaN,0.003409,0.012443,0.007012,0.006253,0.002489,-0.000589,0.0,...,0.012392,0.006388,0.007633,0.005707,0.021359,0.009710,0.007511,0.009325,0.012151,0.014402


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-25      0.000000  0.004164 -0.007899  0.001961  0.022199  0.000000   
2020-08-26     -0.003888  0.016098 -0.019108  0.003914  0.001034 -0.001898   
2020-08-27      0.002342  0.007921 -0.003247  0.003899  0.011364  0.001901   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-25     -0.001242  0.005074 -0.008046 -0.005952  ...  0.000000    0.0   
2020-08-26      0.006219  0.016521 -0.001159 -0.008982  ...  0.012658    0.0   
2020-08-27      0.004944  0.004515 -0.002320  0.015106  ...  0.000000    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-25      0.024793      0.0   0.017241  0.007335  0.000000        NaN   
2020-08-26      0.024194      0.0   0.000000  0.012136  0.021201        NaN   
2020-08-27      0.000000      0.0   0.000000  0.000000  0.000000        inf   

Province_State                  
Admin2         Washakie Weston  
2020-08-25          0.0    0.0  
2020-08-26          0.0    0.0  
2020-08-27          0.0    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-25,0.006423,0.125000,NaN,0.004402,0.021552,0.012015,0.003648,0.000673,-0.001656,NaN,...,0.025189,0.016437,0.017949,0.0,0.090909,0.009308,0.004821,0.044693,0.012026,0.0
2020-08-26,0.003927,0.027778,NaN,0.021703,0.029536,0.012114,0.000519,0.000000,0.001658,NaN,...,0.012285,0.017176,0.010076,0.0,0.166667,0.008420,0.002132,0.016043,0.005484,0.0
2020-08-27,0.015159,0.000000,NaN,0.006740,0.009563,0.011570,0.002076,0.000448,0.000000,NaN,...,0.015170,0.018534,0.004988,0.0,0.000000,0.004771,0.005319,0.042105,0.010000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour      Bibb    Blount Bullock Butler   
2020-08-25         0.0  0.000000     0.0  0.000000  0.000000     0.0    0.0   
2020-08-26         0.0  0.031250     0.0 -0.166667  0.166667     0.0    0.0   
2020-08-27         0.0  0.030303     0.0  0.000000  0.000000     0.0    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun Chambers  Cherokee  ...    Park Platte Sheridan   
2020-08-25      0.043478      0.0  0.000000  ...     NaN    NaN      NaN   
2020-08-26      0.041667      0.0  0.000000  ...     NaN    NaN      NaN   
2020-08-27      0.040000      0.0  0.222222  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-25          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-26          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-27          NaN        NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,0.000963,0.018942,0.009276,0.004412,0.024673,0.000087,0.023864,0.002488,0.005971,0.009058,...,0.003435,0.003157,0.009317,0.019537,0.005861,0.021553,3.011673e-21,0.003259,0.013120,0.019056
2020-08-26,0.001046,0.019061,0.009268,0.020131,0.023068,0.000043,0.026600,0.002769,0.005326,0.010889,...,0.003455,0.003857,0.008733,0.019572,0.005360,0.022897,1.505836e-21,0.003189,0.010592,0.013966
2020-08-27,0.000733,0.018268,0.009291,0.010065,0.029330,0.000022,0.026947,0.003741,0.005151,0.011744,...,0.004057,0.004521,0.010409,0.020903,0.003647,0.024398,7.529182e-22,0.002371,0.015185,0.010263


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,0.003542,0.018382,0.006982,1.173753e-06,0.023888,9.255296e-41,0.032685,0.004165,0.034324,0.000561,...,0.000271,0.013167,0.019668,0.019883,0.007336,0.015454,0.0,0.004952,0.005302,0.038597
2020-08-26,0.003203,0.016913,0.006582,5.868765e-07,0.016846,4.627648e-41,0.034589,0.003831,0.038109,0.000280,...,0.000340,0.006583,0.022158,0.021603,0.059223,0.022764,0.0,0.005169,0.002651,0.058455
2020-08-27,0.001601,0.014160,0.006704,2.934383e-07,0.018132,2.313824e-41,0.030753,0.003657,0.028670,0.000140,...,0.000314,0.003292,0.023107,0.020773,0.029612,0.025981,0.0,0.004370,0.001325,0.057160


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-25,0.004423,0.025000,0.008142,0.000642,0.080665,0.000259,0.023840,0.007388,0.025771,0.008215,...,0.001468,0.011544,0.012356,0.035903,0.016918,0.065506,1.764652e-23,0.004372,0.022208,0.015580
2020-08-26,0.012795,0.023869,0.009124,0.009443,0.040332,0.000129,0.022556,0.007135,0.012910,0.009978,...,0.000734,0.010738,0.013608,0.032257,0.042243,0.061890,8.823260e-24,0.004936,0.025191,0.011822
2020-08-27,0.006466,0.028986,0.009096,0.004722,0.203380,0.000065,0.021801,0.007134,0.013072,0.008873,...,0.004558,0.009529,0.017765,0.034592,0.025077,0.040457,4.411630e-24,0.002924,0.019469,0.006811


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-25,0.007556,0.010995,NaN,0.003219,0.007912,0.008476,0.006563,0.002778,0.002227,5.823215e-50,...,0.006869,0.008886,0.007157,0.004340,0.016260,0.008076,0.005515,0.008368,0.008301,0.008325
2020-08-26,0.012358,0.010866,NaN,0.002076,0.009612,0.008623,0.005011,0.003119,0.001821,2.911608e-50,...,0.010091,0.009672,0.007668,0.003760,0.024162,0.007624,0.005937,0.011897,0.009522,0.010770
2020-08-27,0.013596,0.013502,NaN,0.002743,0.011028,0.007817,0.005632,0.002804,0.000616,1.455804e-50,...,0.011242,0.008030,0.007651,0.004734,0.022761,0.008667,0.006724,0.010611,0.010836,0.012586


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-25      0.005068  0.005073 -0.000935  0.003818  0.015845  0.000439   
2020-08-26      0.000590  0.010585 -0.010022  0.003866  0.008439 -0.000729   
2020-08-27      0.001466  0.009253 -0.006634  0.003882  0.009901  0.000586   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-25      0.000262  0.007933 -0.003385  0.003270  ...  0.002623   
2020-08-26      0.003240  0.012227 -0.002272 -0.002856  ...  0.007640   
2020-08-27      0.004092  0.008371 -0.002296  0.006125  ...  0.003820   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-25      0.000675  0.029381  0.010759   0.009596  0.006785  0.000159   
2020-08-26      0.000338  0.026787  0.005380   0.004798  0.009461  0.010680   
2020-08-27      0.000169  0.013394  0.002690   0.002399  0.004730  0.005340   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-25           -1.0  0.002394 -0.011609  
2020-08-26           -1.0  0.001197 -0.005712  
2020-08-27           -1.0  0.000598 -0.002834  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-25,0.005848,0.069853,NaN,0.003914,0.019102,0.009631,0.002369,0.000357,0.001128,NaN,...,0.018815,0.012703,0.013117,1.686417e-08,0.059691,0.006764,0.004068,0.027057,0.007074,0.003804
2020-08-26,0.004888,0.048816,NaN,0.012809,0.024319,0.010873,0.001444,0.000179,0.001393,NaN,...,0.015550,0.014940,0.011596,8.432087e-09,0.113179,0.007592,0.003100,0.021550,0.006279,0.001902
2020-08-27,0.010023,0.024408,NaN,0.009774,0.016941,0.011222,0.001760,0.000313,0.000697,NaN,...,0.015360,0.016737,0.008292,4.216044e-09,0.056589,0.006182,0.004210,0.031827,0.008140,0.000951


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                          \
Admin2               Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-25      1.477060e-06  0.001669  0.001308  0.003126  0.006259   
2020-08-26      7.385300e-07  0.016459  0.000654 -0.081770  0.086463   
2020-08-27      3.692650e-07  0.023381  0.000327 -0.040885  0.043231   

Province_State                                                            ...  \
Admin2           Bullock        Butler   Calhoun      Chambers  Cherokee  ...   
2020-08-25      0.000355  1.090210e-07  0.022326  3.639234e-11  0.001955  ...   
2020-08-26      0.000178  5.451050e-08  0.031996  1.819617e-11  0.000978  ...   
2020-08-27      0.000089  2.725525e-08  0.035998  9.098084e-12  0.111600  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-25         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-26         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-27         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-25       0.003924      NaN    NaN  
2020-08-26       0.001962      NaN    NaN  
2020-08-27       0.000981      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200827,AK,5801,330419,NaN,43.0,NaN,NaN,NaN,9.0,...,336220,0,0,87c72a3946f8d36630b6b5aa3e426a2222949bcd,0,0,0,0,0,NaN
1,20200827,AL,121023,835786,NaN,1052.0,14005.0,NaN,1430.0,NaN,...,956809,31,0,010113bf224bc02eaf239a639a4a3228098d1ca4,0,0,0,0,0,NaN
2,20200827,AR,58745,631283,NaN,433.0,4104.0,NaN,NaN,99.0,...,690028,7,43,7a29e215de264cc6da8c75df7684b76e27b4bc72,0,0,0,0,0,NaN
3,20200827,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e7544098118fa4e8f18d87f113398f410f7a3fa6,0,0,0,0,0,NaN
4,20200827,AZ,200139,974789,NaN,895.0,21426.0,311.0,NaN,176.0,...,1174928,33,50,34abe3c1f5b6156a42e89f9eca939cd37084ff0a,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200827,AK,5801.0,330419.0,NaN,43.0,NaN,NaN,NaN,9.0,...,336220,0,0,87c72a3946f8d36630b6b5aa3e426a2222949bcd,0,0,0,0,0,NaN
1,20200827,AL,121023.0,835786.0,NaN,1052.0,14005.0,NaN,1430.0,NaN,...,956809,31,0,010113bf224bc02eaf239a639a4a3228098d1ca4,0,0,0,0,0,NaN
2,20200827,AR,58745.0,631283.0,NaN,433.0,4104.0,NaN,NaN,99.0,...,690028,7,43,7a29e215de264cc6da8c75df7684b76e27b4bc72,0,0,0,0,0,NaN
3,20200827,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e7544098118fa4e8f18d87f113398f410f7a3fa6,0,0,0,0,0,NaN
4,20200827,AZ,200139.0,974789.0,NaN,895.0,21426.0,311.0,NaN,176.0,...,1174928,33,50,34abe3c1f5b6156a42e89f9eca939cd37084ff0a,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-27,AK,5801,330419,NaN,43.0,NaN,NaN,NaN,9.0,NaN,...,336220,0,0,87c72a3946f8d36630b6b5aa3e426a2222949bcd,0,0,0,0,0,NaN
2020-08-27,AL,121023,835786,NaN,1052.0,14005.0,NaN,1430.0,NaN,782.0,...,956809,31,0,010113bf224bc02eaf239a639a4a3228098d1ca4,0,0,0,0,0,NaN
2020-08-27,AR,58745,631283,NaN,433.0,4104.0,NaN,NaN,99.0,533.0,...,690028,7,43,7a29e215de264cc6da8c75df7684b76e27b4bc72,0,0,0,0,0,NaN
2020-08-27,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e7544098118fa4e8f18d87f113398f410f7a3fa6,0,0,0,0,0,NaN
2020-08-27,AZ,200139,974789,NaN,895.0,21426.0,311.0,NaN,176.0,NaN,...,1174928,33,50,34abe3c1f5b6156a42e89f9eca939cd37084ff0a,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-27,AK,5801.0,330419.0,NaN,43.0,NaN,NaN,NaN,9.0,NaN,...,336220,0,0,87c72a3946f8d36630b6b5aa3e426a2222949bcd,0,0,0,0,0,NaN
2020-08-27,AL,121023.0,835786.0,NaN,1052.0,14005.0,NaN,1430.0,NaN,782.0,...,956809,31,0,010113bf224bc02eaf239a639a4a3228098d1ca4,0,0,0,0,0,NaN
2020-08-27,AR,58745.0,631283.0,NaN,433.0,4104.0,NaN,NaN,99.0,533.0,...,690028,7,43,7a29e215de264cc6da8c75df7684b76e27b4bc72,0,0,0,0,0,NaN
2020-08-27,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e7544098118fa4e8f18d87f113398f410f7a3fa6,0,0,0,0,0,NaN
2020-08-27,AZ,200139.0,974789.0,NaN,895.0,21426.0,311.0,NaN,176.0,NaN,...,1174928,33,50,34abe3c1f5b6156a42e89f9eca939cd37084ff0a,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE